<img src="https://pbs.twimg.com/profile_images/1092394418135539713/eplLRcDN_400x400.jpg" width=80px style="text-align:right"><h1>The Internet of Production Alliance </h1>

## Data collection program for the [OKW, Map of facilities](https://www.internetofproduction.org/open-know-where)


Author: Antonio de Jesus Anaya Hernandez, DevOps eng. for the IoPA.

Author: The internet of Production Alliance, 2023.

Data was collected by "Makery, gogocarto, and its partners", URL location: https://makery.gogocarto.fr/api

Data source license: ODBL 1.0, Open Data Commons Open Database License

The Open Know Where (OKW) Initiative is part of the Internet of Production Alliance and its members.

License: CC BY SA

![CC BY SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)

Description: Python code for downloading, parsing, filtering, sorting data, exporting the RAW makery locations, and the processed IOPA data as CSV.

In [1]:
# This line installs the required libraries for running the script, uncomment the line:
# !pip install -r requirements.txt

In [2]:
import requests
import pandas as pd

In [3]:
from datetime import datetime
now = datetime.now()

In [4]:
def req_data(url):
    
    response = requests.get(url)

    print(response.status_code)
    
    if response.status_code == 200:
        return response
    else:
        print("Error response: Check URL or internet avalability, and Try again.")
        print(url)

In [5]:
url = "https://makery.gogocarto.fr/api/elements.json"
data = req_data(url).json()
print(type(data))

200
<class 'dict'>


In [6]:
data_ = pd.DataFrame(data)

In [7]:
input_d = pd.DataFrame(data_.data)
print(input_d.shape)

from pandas import json_normalize
input_ = pd.json_normalize(input_d['data'])

(528, 1)


In [8]:
input_.reset_index(drop=True, inplace=True)

In [9]:
input_.to_csv('../data/raw_makery_' + now.strftime("%Y_%m_%d_%H%M") + '.csv')

In [10]:
input_.columns.tolist()

['id',
 'name',
 'sourceKey',
 'createdAt',
 'updatedAt',
 'status',
 'categories',
 'categoriesFull',
 'article_makery',
 'fullAddress',
 'mode_gestion',
 'site_web',
 'email',
 'subscriberEmails',
 'geo.latitude',
 'geo.longitude',
 'address.customFormatedAddress',
 'description_courte',
 'surface',
 'telephone',
 'images',
 'openHours.Mo',
 'openHours.Tu',
 'openHours.We',
 'openHours.Th',
 'openHours.Fr',
 'address.streetNumber',
 'address.streetAddress',
 'address.addressLocality',
 'address.postalCode',
 'address.addressCountry',
 'affiliation',
 'description',
 'openHours.Sa',
 'openHours.Su']

In [11]:
transform = input_.rename(columns={'id': 'makery_id', 'status': 'makery_status', 'site_web': 'url', 'geo.latitude':'latitude', 'geo.longitude': 'longitude'})

In [12]:
#transform['makery_url'] = 'https://makery.gogocarto.fr/map#/fiche/{name}/{id}/'.format(name=transform.name.replace(' ','-'), id=transform.makery_id)

transform['makery_url'] = transform.apply(
    lambda row: 'https://makery.gogocarto.fr/map#/fiche/{name}/{id}/'.format(
        name=str(row['name']).replace(' ', '-'),
        id=row['makery_id']
    ),
    axis=1
)



In [13]:
transform = transform[transform['makery_status'] != 'closed']

In [14]:
transform = transform[transform['makery_status'] != 'planned']

In [15]:
transform['address_composed'] = transform['address.streetNumber'].astype(str) + ', ' + transform['address.streetAddress'].astype(str) + ', ' + transform['address.addressLocality'].astype(str) + ', ' + transform['address.postalCode'].astype(str) + ', ' + transform['address.addressCountry'].astype(str)

In [16]:
drops = [
 'sourceKey',
 'categoriesFull',
 'article_makery',
 'fullAddress',
 'mode_gestion',
 'subscriberEmails',
 'address.customFormatedAddress',
 'description_courte',
 'surface',
 'telephone',
 'images',
 'openHours.Mo',
 'openHours.Tu',
 'openHours.We',
 'openHours.Th',
 'openHours.Fr',
 'address.streetNumber',
 'address.streetAddress',
 'address.addressLocality',
 'address.postalCode',
 'address.addressCountry',
 'affiliation',
 'description',
 'openHours.Sa',
 'openHours.Su']

In [17]:
output = transform.drop(columns=drops)

In [18]:
output.columns.tolist()

['makery_id',
 'name',
 'createdAt',
 'updatedAt',
 'makery_status',
 'categories',
 'url',
 'email',
 'latitude',
 'longitude',
 'makery_url',
 'address_composed']

In [19]:
output.to_csv('../data/iopa_makery_' + now.strftime("%Y_%m_%d_%H%M") + '.csv')

In [20]:
print("OKW entries: {r[0]}, columns = {r[1]}".format(r=output.shape))

OKW entries: 528, columns = 12
